In [21]:
import json
import os
import shutil
import pandas as pd
pd.set_option('display.max_columns', None)

# Collect results for a set of folders

In [44]:
%%time

save = False

df = pd.DataFrame(columns=['folder', 'ae_hidden_sizes', 'ae_type', 'batch_size', 'best_acc', 'best_epoch',
       'best_val_loss', 'clf_hidden_sizes', 'clf_nonlinearity', 'clf_type',
       'encoding_size', 'epoch_patience', 'epochs_tained_for', 'img_size',
       'learning_rate', 'momentum', 'num_epochs', 'ra_hidden_size',
       'ra_network_type', 'ra_nonlinearity', 'ra_num_layers', 'ra_type',
       'time_sec', 'weight_decay'])

# Assumes your experiment results are stored in {i}.json files under {parent_expdir}/expdir/{res_folder}
# Collects over all expdirs in the iterable below
parent_expdir = 'exp'
res_folder = 'res'
expdirs = ['01_e2e']

for expdir in expdirs:
    for f in os.listdir(os.path.join(parent_expdir, expdir, res_folder)):
        if f[-4:] == 'json':
            with open(os.path.join(parent_expdir, expdir, res_folder, f)) as fp:
                x = json.load(fp)
                x['folder'] = expdir
                del x['losses']
                x['id'] = int(f.split('.')[0])
                x['clf_hidden_sizes'] = str(x['clf_hidden_sizes'])
                x['ae_hidden_sizes'] = str(x['ae_hidden_sizes'])
            df = df.append(pd.DataFrame(x, index=[0]))

if save:
    df.to_csv(f'summary_{"_".join(expdirs)}.csv', index=False)

CPU times: user 12.4 s, sys: 112 ms, total: 12.5 s
Wall time: 12.7 s


# View best results

In [ ]:
best = df.sort_values(by='best_acc', ascending=False).head(10)
best

# Which models did not stop early?

In [ ]:
not_stopped = df[df.num_epochs <= df.epochs_tained_for].sort_values(by='best_acc', ascending=False)
not_stopped

# Get a list of hyperparameters to retrain models that did not stop early

In [ ]:
from ast import literal_eval
ids = [134]

hyperparams = []

for idx in ids:
    x = not_stopped[not_stopped.id == idx].drop(columns=['best_acc',
        'best_epoch','best_val_loss', 'epochs_tained_for', 'folder',
        'time_sec']).iloc[0].to_dict()
    x['ae_hidden_sizes'] = literal_eval(x['ae_hidden_sizes'])
    x['clf_hidden_sizes'] = literal_eval(x['clf_hidden_sizes'])
    hyperparams.append(x)

hyperparams